# Advanced Julia: How to not write slow code

Tommy Hofmann (University of Siegen) — 9/7/2021

## What we will talk about

- Why your code makes Julia sad
- How to notice it
- Common pitfalls and how to avoid them

## What we will not talk about

- General programming tipps

# Why your code makes Julia sad

In Julia, every value/object has a type:



In [1]:
typeof(1), typeof("123")

(Int64, String)

If we assign a value to a variable, then this variable also has a type (not quite correct, but this simplified view is good enough for us).

In [2]:
x = 1
typeof(x)

Int64

One reason Julia is fast, is that whenever one has a call of the form `f(x)`, a special version of `f` for the given type of `x` is compiled.

In [3]:
function f(x)
    return x * x + 1
end

f (generic function with 1 method)

In [4]:
@code_native syntax=:intel debuginfo=:none f(1)

	.text
	imul	rdi, rdi
	lea	rax, [rdi + 1]
	ret
	nop	dword ptr [rax]


In [5]:
@code_native syntax=:intel debuginfo=:none f(1.0)

	.text
	vmulsd	xmm0, xmm0, xmm0
	movabs	rax, offset .rodata.cst8
	vaddsd	xmm0, xmm0, qword ptr [rax]
	ret
	nop	word ptr cs:[rax + rax]


Let's look at the following example:

In [6]:
function g(y::Float64)
    if y < 0.5
        x = 1
    else
        x = 1.0
    end
    return f(x)
end

g (generic function with 1 method)

- If we call `g(::Float64)`, will `f` be called with an argumet of type `Int` or type `Float64`?
- We cannot say by just looking at the input type!
- If we cannot tell, then also the Julia "compiler" cannot tell.

**Julia has to determine at runtime what to do and cannot produce efficient code!**

Is it really that bad?

In [7]:
function h(x)
    y = x[1]
    for i in 2:length(x)
        y = y + x[i]
    end
    return y
end

using BenchmarkTools

x = Any[i for i in 1:1_000_000]
@btime h($x)
    
x = Int[i for i in 1:1_000_000]
@btime h($x)

  17.759 ms (999969 allocations: 15.26 MiB)
  557.046 μs (0 allocations: 0 bytes)


500000500000

This is a 20x slowdown!

What is happening here?

- In the first version, we have `typeof(x) == Vector{Any}`, so just by looking at the type, Julia has no idea what `x[i]` will be. It could be anything!

- In the second version, we have `typeof(x) == Vector{Int}`, so just by looking at the type, Julia knows that `x[i]` will have type `Int` and it knows how to produce fast code.

I am not a Julia wizard, how do I recognize this?

`@code_warntype` to the rescue!

In [8]:
function g(y::Float64)
    if y < 0.5
        x = 1
    else
        x = 1.0
    end
    return f(x)
end

g (generic function with 1 method)

In [9]:
@code_warntype g(1.0)

Variables
  #self#::Core.Const(g)
  y::Float64
  x::Union{Float64, Int64}

Body::Union{Float64, Int64}
1 ─      Core.NewvarNode(:(x))
│   %2 = (y < 0.5)::Bool
└──      goto #3 if not %2
2 ─      (x = 1)
└──      goto #4
3 ─      (x = 1.0)
4 ┄ %7 = Main.f(x)::Union{Float64, Int64}
└──      return %7


Julia tells us that it only knows that `x` can be `Float64` or `Int`, so `x::Union{Float64, Int}`. Because this is dangerous for performance, it is marked red.

## The mantra of fast Julia code

Write **type stable** code, which means that
- The type of the variables inside functions is determined by the type of the input arguments alone.
- The return type of functions should be determined by the type of the input arguments alone.

## Common pitfalls and how to avoid them

###  1. Type instable functions

### Example

The following function is not type stable.

In [10]:
function negative_or_positive(x::Int)
    if x < 0
        return "negative"
    elseif x > 0
        return "positive"
    else
        return false
    end
end

negative_or_positive (generic function with 1 method)

In [11]:
@code_warntype negative_or_positive(1)

Variables
  #self#::Core.Const(negative_or_positive)
  x::Int64

Body::Union{Bool, String}
1 ─ %1 = (x < 0)::Bool
└──      goto #3 if not %1
2 ─      return "negative"
3 ─ %4 = (x > 0)::Bool
└──      goto #5 if not %4
4 ─      return "positive"
5 ─      return false


The type of the output depends not only on the type of the input, but on its value.
        
This makes the function not type stable.

### Solution
Don't write type instable code.

### Another example

In [2]:
function gimme_square(x::Int; as = "int")
    y = x^2
    if as == "int"
        return y
    elseif as == "string"
        return string(y)
    else
        error("what")
    end
end

gimme_square (generic function with 1 method)

In [3]:
gimme_square(2, as = "int")

4

In [4]:
gimme_square(2, as = "string")

"4"

This is bad. The type of the return value depends on the *value* of `as`.

### Solution

In [6]:
gimme_square(as::Type{Int}, x::Int) = x^2

gimme_square(as::Type{String}, x::Int) = string(x^2)

gimme_square (generic function with 3 methods)

In [7]:
gimme_square(Int, 2)

4

In [8]:
gimme_square(String, 2)

"4"

(Can also be writen in one function with `Type{T}` argument and checking `as === Int` or `as === String`.)

## 2. Variables should not change type

Similar to type stability but refers more to the internal structure of the function.

### Example

In [45]:
using Oscar

function add_powers(x::RingElement, n) # compute x^1 + x^2 + ... + x^n
  z = 0
  for i in 1:n
    z = z + x^i
  end
  return z
end

add_powers (generic function with 1 method)

In [13]:
@code_warntype add_powers(QQ(1), 10)

Variables
  #self#::Core.Const(add_powers)
  x::fmpq
  n::Int64
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  z::Union{Int64, fmpq}
  i::Int64

Body::Union{Int64, fmpq}
1 ─       (z = 0)
│   %2  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = z::Union{Int64, fmpq}
│   %11 = (x ^ i)::fmpq
│         (z = %10 + %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return z


We can ignore the yellow. But the red is pretty *bad*. The problem is that at the beginning we have `z::Int`, while in the loop the variable `x` is added, which turns it into the same type of `x`, that is, `z::fmpq`.

### Solution

Don't do it! Use the zero of the right type to accumulate into:

In [14]:
function add_powers_better(x::RingElement, n)
  z = zero(parent(x))
  for i in 1:n
    z = z + x^i
  end
  return x
end

add_powers_better (generic function with 1 method)

In [15]:
@code_warntype add_powers_better(QQ(1), 10)

Variables
  #self#::Core.Const(add_powers_better)
  x::fmpq
  n::Int64
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  z::fmpq
  i::Int64

Body::fmpq
1 ─ %1  = Main.parent(x)::Core.Const(Rational Field)
│         (z = Main.zero(%1))
│   %3  = (1:n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = z::fmpq
│   %12 = (x ^ i)::fmpq
│         (z = %11 + %12)
│         (@_4 = Base.iterate(%3, %10))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return x


## 3. Be explicit about the element type of arrays

### Example

In [16]:
function get_powers(x::RingElement, i::Int)
  r = []
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers (generic function with 1 method)

In [17]:
@code_warntype get_powers(ZZ(3), 2)

Variables
  #self#::Core.Const(get_powers)
  x::fmpz
  i::Int64
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  r::Vector{Any}
  j::Int64

Body::Any
1 ─       (r = Base.vect())
│   %2  = (1:i)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (j = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = r::Vector{Any}
│   %11 = (x ^ j)::fmpz
│         Main.push!(%10, %11)
│         (@_4 = Base.iterate(%2, %9))
│   %14 = (@_4 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄ %18 = Base.getindex(r, i)::Any
└──       return %18


The problem is that `r = []` is the same as `r = Any[]`. Thus when accessing `r`, julia does not know what the return type will be.

(It is not marked red, because we explicitely asked for it.)

### Solution
Always annotate the element type of arrays.

In [18]:
function get_powers_better(x::RingElement, i::Int)
  r = typeof(x)[]
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers_better (generic function with 1 method)

In [19]:
@code_warntype get_powers_better(ZZ(3), 2)

Variables
  #self#::Core.Const(get_powers_better)
  x::fmpz
  i::Int64
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  r::Vector{fmpz}
  j::Int64

Body::fmpz
1 ─ %1  = Main.typeof(x)::Core.Const(fmpz)
│         (r = Base.getindex(%1))
│   %3  = (1:i)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (j = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = r::Vector{fmpz}
│   %12 = (x ^ j)::fmpz
│         Main.push!(%11, %12)
│         (@_4 = Base.iterate(%3, %10))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄ %19 = Base.getindex(r, i)::fmpz
└──       return %19


Alternatively:

In [20]:
function get_powers_better2(x::T, i::Int) where T <: RingElement
  r = T[] # typeof(x)[]
  for j in 1:i
    push!(r, x^j)
  end
  return r[i]
end

get_powers_better2 (generic function with 1 method)

But what should I do if I don't know the element of the array yet?

### Example

In [21]:
import Oscar: Ring

function count_to_10(R::Ring)
  # compute r = [R(1), R(2), ..., R(10)]
  r = []
  for i in 1:10
    push!(r, R(i))
  end
  return r[2]
end

count_to_10 (generic function with 1 method)

### Solution

Initialize the array with the intented element type.

In [23]:
function count_to_10_better(R::Ring)
  r = elem_type(R)[]
  for i in 1:10
    push!(r, R(i))
  end
  return r[2]
end

count_to_10_better (generic function with 1 method)

The functions `elem_type`, `parent_type`, `dense_matrix_type` help with manipulations on the type side.

# 4. Properly type the fields of your own types

Assume we want to implement an efficient version of two by two matrices
$$ \begin{pmatrix} a & b \\ c & d \end{pmatrix} $$
We want a version which works over any ring!

In [25]:
mutable struct TwoByTwoMatrix
    a
    b
    c
    d
end

my_det(A) = A.a * A.d - A.b * A.c

my_det (generic function with 1 method)

In [26]:
A1 = TwoByTwoMatrix(ZZ(2), ZZ(1), ZZ(0), ZZ(2))

TwoByTwoMatrix(2, 1, 0, 2)

In [27]:
my_det(A1)

4

In [28]:
@code_warntype my_det(A1)

Variables
  #self#::Core.Const(my_det)
  A::TwoByTwoMatrix

Body::Any
1 ─ %1 = Base.getproperty(A, :a)::Any
│   %2 = Base.getproperty(A, :d)::Any
│   %3 = (%1 * %2)::Any
│   %4 = Base.getproperty(A, :b)::Any
│   %5 = Base.getproperty(A, :c)::Any
│   %6 = (%4 * %5)::Any
│   %7 = (%3 - %6)::Any
└──      return %7


Not properly typing the fields is good for quick prototyping, but should not be done in the final version. Our version of `TwoByTwoMatrix` is the same as

In [29]:
mutable struct TwoByTwoMatrix
    a::Any
    b::Any
    c::Any
    d::Any
end

When typing the fields, a common mistake is to use abstract types:

In [30]:
mutable struct TwoByTwoMatrix2
    a::RingElement
    b::RingElement
    c::RingElement
    d::RingElement
end

In [31]:
A2 = TwoByTwoMatrix2(ZZ(2), ZZ(1), ZZ(0), ZZ(2))

TwoByTwoMatrix2(2, 1, 0, 2)

In [32]:
my_det(A2)

4

In [33]:
@code_warntype my_det(A2)

Variables
  #self#::Core.Const(my_det)
  A::TwoByTwoMatrix2

Body::Any
1 ─ %1 = Base.getproperty(A, :a)::RingElement
│   %2 = Base.getproperty(A, :d)::RingElement
│   %3 = (%1 * %2)::Any
│   %4 = Base.getproperty(A, :b)::RingElement
│   %5 = Base.getproperty(A, :c)::RingElement
│   %6 = (%4 * %5)::Any
│   %7 = (%3 - %6)::Any
└──      return %7


- While `RingElement` is a bit more precise than `Any` it is still an *abstract* type. It does not help at all.
- When Julia sees `A.a` it still does not know which type this object will be.
- Could be an integer, rational, polyomial, ..., any ring element.

In [34]:
mutable struct TwoByTwoMatrix3{T <: RingElement}
    a::T
    b::T
    c::T
    d::T
end

In [35]:
A3 = TwoByTwoMatrix3(ZZ(2), ZZ(1), ZZ(0), ZZ(2))

TwoByTwoMatrix3{fmpz}(2, 1, 0, 2)

- Every instance of `TwoByTwoMatrix3` comes with specific `T`.
- In our case `typeof(ZZ(1)) == fmpz`, so that `T == fmpz`.

In [36]:
my_det(A3)

4

In [37]:
@code_warntype my_det(A3)

Variables
  #self#::Core.Const(my_det)
  A::TwoByTwoMatrix3{fmpz}

Body::fmpz
1 ─ %1 = Base.getproperty(A, :a)::fmpz
│   %2 = Base.getproperty(A, :d)::fmpz
│   %3 = (%1 * %2)::fmpz
│   %4 = Base.getproperty(A, :b)::fmpz
│   %5 = Base.getproperty(A, :c)::fmpz
│   %6 = (%4 * %5)::fmpz
│   %7 = (%3 - %6)::fmpz
└──      return %7


Everything is nicely typed :)

In [38]:
@btime my_det($A1)
@btime my_det($A2)
@btime my_det($A3)

  73.321 ns (3 allocations: 48 bytes)
  82.980 ns (3 allocations: 48 bytes)
  50.399 ns (3 allocations: 48 bytes)


4

- So roughly a 20% speedup by doing almost nothing
- Important for workhorse functions and tight loops
- Big(gest) problem with instabilities is that they propagate, infesting all the callers.

Equally bad as `RingElement` is `Vector` or `Array`. Correct way would be `Vector{fmpz}`/`Vector{T}` or `Array{fmpz, 3}`/`Array{T, 3}`.

# 5. If you have to work with instabilities, help the compiler
There are situations, where writing type unstable is unavoidable. Here is an example

In [39]:
mutable struct NewType
    x
end

In [40]:
R = NewType(2); @code_warntype R.x

Variables
  #self#::Core.Const(getproperty)
  x::NewType
  f::Symbol

Body::Any
1 ─ %1 = Base.getfield(x, f)::Any
└──      return %1


In [48]:
function something(R::NewType)
  # for some reason we know that in this situation R.x
  # will always be an Int 
  y = R.x::Int
  return y^2
end

something (generic function with 1 method)

In [49]:
R = NewType(2); @code_warntype something(R)

Variables
  #self#::Core.Const(something)
  R::NewType
  y::Int64

Body::Int64
1 ─ %1 = Base.getproperty(R, :x)::Any
│        (y = Core.typeassert(%1, Main.Int))
│   %3 = y::Int64
│   %4 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %5 = (%4)()::Core.Const(Val{2}())
│   %6 = Base.literal_pow(Main.:^, %3, %5)::Int64
└──      return %6


Notice the `typeassert`!

# 6. Digging deep: Cthulhu

Demo for the package Cthulhu's `@descend` macro.

# 7. Avoid unnecessary allocations

In [50]:
function allocate_much(n::Int, k::Int) # Compute [1,...,n,1^2,2^2,...,n^2,...,1^k,2^k,...,n^k]
    x = Int[]
    for i in 1:k
        x = vcat(x, [j^i for j in 1:n])
    end
end

@btime allocate_much(1000, 1000);

  650.116 ms (2999 allocations: 3.74 GiB)


In [51]:
function allocate_less(n::Int, k::Int)
    x = Int[]
    for i in 1:k
        for j in 1:n
            push!(x, j^i)
        end
        #Alternative: append!(x, (j^i for j in 1:n))
    end
    return x
end

@btime allocate_less(1000, 1000);

  13.827 ms (20 allocations: 9.00 MiB)


This is a 60x speedup by doing almost nothing

- Use `!`-functions whenever possible, in particular when manipulating arrays.
- Use inplace operations like `mul!` or `add!` when doing computation with ring elements (but be careful).

# 8. Random tipps
- Skim over https://docs.julialang.org/en/v1/manual/performance-tips/
- For deeply nested code (or keyword arguments), use https://github.com/JuliaDebug/Cthulhu.jl
- If your functions allocate a lot of memory (when timed with `@btime` or `@time`), chances are high that there is type instability. Check with `@code_warntype`.
- Use `@benchmark` and `@btime` from the BenchmarkTools package for serious benchmarking.